<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install fsspec==2023.6.0
!pip install --upgrade transformers tokenizers
# !pip uninstall -y torch torchtext
# !pip install torch==2.0.1+cu117 torchtext==0.15.2 -f https://download.pytorch.org/whl/torch_stable.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [ ]:
# from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import GPT2Tokenizer
print(torch.__version__)

2.6.0+cu124


In [ ]:
from datasets import load_dataset

# This should load without issues
dataset = load_dataset("wikitext", "wikitext-2-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
type(dataset)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

TypeError: can only concatenate list (not "dict") to list

In [ ]:
inputs = tokenized['train']['input_ids']
print(type(tokenized['train']))

all_input_ids = sum(inputs, [])


# block_size = 128
# num_blocks = len(all_input_ids) // block_size

# input_blocks = [
#     all_input_ids[i * block_size : (i + 1) * block_size]
#     for i in range(num_blocks)
# ]

# # Optional: Convert to PyTorch tensors
# input_tensor = torch.tensor(input_blocks)

# print(input_tensor.shape)  # (num_blocks, block_size)
inputs[:6]

In [ ]:
# tokens['input_ids'].shape,tokens['attention_mask'].shape
# seqs = [(tokens['input_ids'][i:i+7], tokens['input_ids']) for i in range(tokens['input_ids'].shape[0])]
# seqs[0]
print("hi")

{'input_ids': tensor([[50256, 50256, 50256,  ..., 50256, 50256, 50256],
        [  796,   569, 18354,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256],
        ...,
        [28036,   433,   550,  ..., 50256, 50256, 50256],
        [ 8070,  3491, 17783,  ..., 50256, 50256, 50256],
        [50256, 50256, 50256,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}

In [ ]:
class LMModel1(nn.Module):
  def __init__(self, vocab_sz, n_hidden):
    self.i_h = nn.Embedding(vocab_sz, n_hidden) # Embedded layer. Maps inputs (vocab) to vector
    self.h_h = nn.Linear(n_hidden, n_hidden)    # Hidden layer
    self.h_o = nn.Linear(n_hidden, vocab_sz)    # Output layer

  def forward(self, x):
    h = F.relu(self.h_h(self.i_h(x)))
    return self.h_o(h)